Import our dependencies and the Minitorch from my github

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sys
import os
from tqdm import tqdm


from engine.minitorch import Tensor,optim

I will put a seed right now for the reproducibility of the project.

In [3]:
np.random.seed(3329)

This are the variables that we need in order to create the data folder with the dataset inside,because I can't upload the repository with the dataset of Shakespeare pre-charged.

In [4]:
data_directory = "data"
file_path = os.path.join(data_directory, "input.txt")
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

This will create a data folder if it doesn't exists

In [5]:
if not os.path.exists(data_directory):
    
    os.makedirs(data_directory)

if the inside of the data folder that we just created is empty we throw a request to the url for the dataset of Shakespeare of the github of Karpathy,raise for status to inform us what happened with the request and then open it indicating to the computer that need to transform it binary to avoid missinterpretations of characters and then finally write it down to the "input.txt" inside our data folder

In [6]:
if not os.path.exists(file_path):
    
    response = requests.get(url)
    
    response.raise_for_status()


    with open(file_path, "wb") as f:

        f.write(response.content)

        

Here what we do is open the input.txt of Shakespeare preaviously stored in the data folder and read it,the encoding utf-8 is the standard to read special characters like emojis,commas,dots,etc.\
Then the text variable is just all that concatenated ,like a list of all Shakespeare.

In [7]:
with open(file_path, 'r', encoding='utf-8') as f:

    text = f.read()

First we convert the text into a set,this will give us every character inside the text.\
Then we transform that set into a list to be able to sorted in order that the same character never change its index.\
Lastly we can see the len of the vocabulary and print it.

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


To tokenize them we just create two dictionaries, String to Integer(stoi) and Integer to String(itos).

One will have as key_value the character and as the value the index of that character in the list of chars.

And the other one will have its key_value the index and as value the character of that index in the list of chars.

This is exactly what we want because we use the list of characters and its indexes to transform them into integers and vice versa.

In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}

itos = {i:ch for i,ch in enumerate(chars)}

Now here is the actual encoding and decoding.\
We gonna use lambda function because this is simple and with lambda we avoid writing a def just for this.

encoder = here we pass to lamba a string(s) then we ask for its correspondent value inside the stoi dictionary.

decoder = this will receive a list of numbers,search what character correspond to that integer inside of itos and concatenate them into a string.

In [10]:
encoder = lambda s:[stoi[c] for c in s]

decoder = lambda l: ''.join([itos[i]for i in l])

Now that we got our encoder and decoder que can transform the text into a numpy array.

In [ ]:
all_data = np.array(encoder(text), dtype=np.int64)

Here we gonna take the 90% of the total data(len of total_data * 0.9).\
And then we use that number to divide the data into train_data and test_data.

train_data being the first 90% of the data.\
test_data being the last 10% of the data.

In [ ]:
n = int(0.9 * len(all_data))

train_data = all_data[:n]
test_data = all_data[n:]

This is momentary and its something that you can change to have a different speed or precision in training.\
I realize that I need this 2 variables because of the creation of the parameters for the feed-foward network later on in the creation of the foward pass functions,so I decide to put this right up here(In the training preparation you can change the values according to how you want to train the model).

In [ ]:
block_size = 8
batch_size = 4

Here is the function to create batches because for training you can't pass all the data at once,explanation = Self-explanatory, just don't.

For that reason we create this function passing as arguments the split which is a string.\
The batch_size which is one of the dimensions and the block_size which is the other dimension.

First we declare the variable data, this variable will be equal to the train_data if when we pass the split(the string parameter) we said "train" otherwise the variable data will be equal to the test_data.

Now we gonna create the randomness in the batches in order to train better the model,using numpy,we tell it to give us a random number between 0 and the total of the data minus the block size so it doesn't give us por example the last 2 tokens in the data and broke our block size.\
And then we also pass the batch_size to tell numpy that we want more than a single number.

Next we gonna create our Tensors2D(block_size,batch_size).\
To create our inputs (x) we need to put them in a stack in order to create the mesh/matrix so using and excel sheet as analogy you can interpret the block size as the how large will we the rows and the batch_size as how large will be the columns(for example, if we pass block_size=8 and batch_size=4 the tensor that we gonna get is a 4x8 matrix,4 batches with a lenght of 8 values each).\
Once acquire the inputs, the targets(y) follow the same creation logic but +1.It will be the same size as the tensor "X".
Example: if tensor start with something like  43,56,21 then tensor Y will be like 56,21,33 .

Lastly we just return both the inputs and targets tensors.

In [ ]:
def get_batches(split,batch_size,block_size):

    data = train_data if split == "train" else test_data

    idx = np.random.randint(0,len(data) - block_size,batch_size)

    x = np.stack(data[i:i+block_size] for i in idx)

    y = np.stack(data[i + 1 : i + block_size + 1] for i in idx)

    return x, y

Here we have our variables to be able to create the table of embedding.\
We will use this to transform the 65 posibles tokens into vectors of random values(one vector per char) to be able to do maths operations with the vectors.

First we take the vocab_size.\
Then we assing a variable called numbers of embedding which is arbitrary,for a model like this I recommend 128 because is a tiny model,every of the 128 will give to the model context of what is about the token, for example, if the token 5 represent the letter "a" the table of embedding with each of the 128 will give us an adjective of it: "vocal","minuscule".


Then we initialize the variable table creating a tensor2D of random numbers,each row will represent 1 token(65 row because of the vocab_size) and every column is value of embedding for that token(128 embeddings per token).Creating a tensor2D of (65,128).\
We just multiply those values by the Xavier formula that is commonly used for Softmax/Sigmoid/Tanh because the randomness is too high for the model to start learning right away,it could probably(and luckily if doesn't explode)take a lot of epochs start learning.

By the way,we created this Variables outside the get_embeddings function because we need to train this random values,if we initialize them inside the function they will be created every time we call the function and never train.

In [ ]:
vocab_size = vocab_size # Just to bring it closer we already know that they are 65 characters

n_embd = 128


token_table = np.random.randn(vocab_size,n_embd) * np.sqrt(2/n_embd)

This short function is the embedding of our tokens,is short because Numpy does the hard lifting for us creating the new tensor and all that.

When we call the function we pass our batches of tokens for example (4x8) and we also pass the table of embeddings of random values(65,128).\
What Numpy does is create a new Tensor, but this tensor is a Tensor3D.\
When we said use the inputs as index of table we practically say,from the embeddings table put in a new tensor the rows of every value in the block_size of the batch_size giving us a Tensor3D of (4,8,128).\
In an excel sheet example in this new tensor we got: the 128 embeddings columns(n_embd=128) for each row of the 8 rows(block_size=8) and once we got that tensor2D we need to stack 4 tensor2D like those(batch_size=4) on top of each other.

In [ ]:
#  def get_embeddings(xb,token_table):
#      return token_table[xb]

Same as the token_table we iniliazite a tensor2D but instead of using the vocab_size we use the block_size(Also multiplying by the Xavier formula).

In [ ]:
pos_table = np.random.randn(block_size,n_embd) * np.sqrt(2/n_embd)

I encourage you to learn the theory behind this positional encoding,is facinating see how words can follow a sinusoidal wave to know time and position of each embedding(maybe in future projects I'll implement this advance,more mathematical version on a transformer,which is the original way suggested by the paper "Atenttion is all you need",extrapolation, "infinite" positions),For now I'll stick with the GPT way(Learned positional embeddings)which is just another tensor like the "embeddings_table" that add time and position with random numbers that later on the model will learn.\
Because the model without the positional encoding will think that "The dog bites the man" is the same as "The man bites the dog".


Here we take as parameters the stack batch(4,8) the token table and pos_table that we previusly have created.

For future operations I'll take the shape of xb .\
Now to make things more modular just in case that in the future I want a block_size of 16 instead of one of 8 like right now,we create an numpy array with the length of our block_size(T=8).

Then we have the token_table(65,128) but we need a tensor3D of (4,8,128 like the token_embeddings of the previous function).\
so we use the same fancy indexing trick to give us the tensor3D.(after a conclusion I find the previous functon redundant and I comment it, so it doesn't affect when running all the cells,just in case;But I will leave it right there because I see it as a sign of learn not as a mistake).

Then this next part is just to "name" put "labels" to every row in the pos_table,for example the first row of pos_table will be named position:0(this a analogy of how the model will think) and the next row position:1, this will continue until position:7 because the block_size only have 8 positions.\
This will give to the model a perception of time/space because this value will be train to tell the posible position of the token after seeing all the Shakespeare train data.

Next and finally is just the summ of the token_embd which give us "characteristics" of the token with the pos_embd that will give us "time/space" of the token,if you are wondering how two tensor with different sizes can do an addition to each other,the answer is : Because of numpy,numpy is doing the broadcasting so we don't have problems with the dimension,he es just doing again and again the addition of the tensor2D(pos_embd) the amount of batches of the tensor3D(4),so we ended up with a (4,8,128) tensor.

In [ ]:
def positional_encoding(xb,token_table,pos_table):

    B,T = xb.shape

    pos_idx = np.arange(T)

    token_embd = token_table[xb] # .data because that is the numpy array inside the Tensor of my Minitorch

    pos_embd = pos_table[pos_idx]

    x = token_embd + pos_embd

    return x

I'm using my minitorch but the "new parts" of the Transformer(positional encoding,multi-head atenttion,etc) I'll do it manual(including backprop).\
So for that reason I'll need a Softmax function.\
Quick recap(I've been explaining it in every project):

1 - Take the max number of the array so it doesn't explode doing the exponentiation.\
2 - Before the exponentiation to the array subtract the max value of the array to the array ,then exponentiate it.\
3 - The Result is just the array exponentiated divided the summatory of the array exponentiated.

Btw Softmax is mostly applied in the last dimension so the axis = -1 should be a standard and is how I did it in my Minitorch.\
(Softmaxx with double "xx" is on purpose to make sure that it doesn't do a conflict with the Softmax function of my minitorch).

In [ ]:
def Softmaxx(inputs):

    max_num = np.max(inputs, axis = -1, keepdims=True)

    scores = np.exp(inputs - max_num)

    out = scores / np.sum(scores, axis = -1, keepdims=True)

    return out

And I'll do its backpropagation that is something that I also did in my Minitorch.\
Its just using the trick with the formula of the Jacobian.

I'll not go deep into mathematical reasons but the derivative of Softmax is just the out of the foward pass of the softmax multiplied by the substraction of the summatory of the gradient multiplied by the out of the foward to the gradient.

In [ ]:
def backprop_Softmax(out,gradient):

    summatory_gradout = np.sum(gradient * out, axis = -1, keepdims=True)

    dSoftmax = out * (gradient - summatory_gradout)

    return dSoftmax

Here,is important to first of all create our parameters (Query, Key, Value) outside the function.\
To create them we just gonna use the embeddings dimmensions for something that I'll explain in the multi_head_atenttion function.
Like always when you create a tensor with random values you need to multiply them by the Xavier or He init formula(The one that comes better for the reason of what you gonna use that tensor with random values).

Note:The Wo variable is also important and is written in the paper "Atenttion is all you need" is like a "mixxer" of the reponses of the heads to make the output have more coherence,but its responsabilities are not as "important" as the other 3,just for that reason to clarify the panoram I'll treat it different.

In [ ]:
Wq = np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd)
Wk = np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd)
Wv = np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd)

Wo = np.random.randn(n_embd, n_embd) * np.sqrt(1 / n_embd)

Here is the heart of the transformer,The MHA(Multi Head Atenttion),I've divide it by sections for me to be easier to explain:

Section 0: This is just preparation,we take the shape of our inputs and create the size of every of the 4 heads,size which will be 32 embeddings each head.


Section 1: Here we combine the data of our input with the data of ours parameters just to differentiate them of the function of what each one does.\
Creating the variables Q (query), K(key), V(value).


Section 2: This step is simply to organize them in orden to be able to do its respective function down the road.\
First we take the Q,K,V data from the previous section and reshape them to become a tensor4D(4,8,4,32) and then transpose them to be able to work with the 32 embeddings(1 head) of each one of the token inside the block_size.


Section 3: Here is when we use the differentiation of each one.\
Commonly I think about the query as the "signal" that the token who wants to predict the next one emits to start searching in a library(the "What i want").\
The key I interpret them as Label/Name of the books (each token have 4 books describing it,the heads).\
And finally the value as the content of the book.

Our purpose is create 4 new books(One Token) so in order to create them we going to search in the library and take the content that we want for those next new 4 books.

So in this section we compare every label of the book in the library with what we want to know how much of every book we going to take.\
The formula will be Q*K.transpose for that searching,transpose because (excluding the dimension that we don't care right now) we got Q=(8,32) and K=(8,32) so in orden to do the matmul we transpose the K ->(32,8) and now we can start "searching" (8,32)*(32,8).

Finally for this section, we going to do the scaling to the weights(wei), This is just for the Softmax so it doesn't explodes with numbers to high.\
Like lowering the volume of the values so Softmax doesn't get stunned.


Section 4: We need to do Softmax in order to be able to know how many of each of the heads we need to take to predict the next token correctly.\
For that reason we do Softmax in the last dimension,but something that happens when we do the Softmax is that the Softmax also takes in consideration the next tokens(the ones that we want to predict),for example,if we are in position 5 and we want to predict the token of position 6 we going to do a Softmax to all the tokens from position 0 to position 7 and if you notice that is an issue because with Softmax the model will cheat because it will know already what token is next.\
The solution for that is an Atenttion mask,what is that you will ask,well you can see it as phantom tensor that we put on top of our tensor of tokens,this atenttion mask will have a triangular form that we gonna fill with 1s an the rest with either "-inf" or a number extremely low.\
What this will do?This atenttion mask will prevent the model to cheat because the mask will replace our tensor with numbers in a form that every time we want to predict the next token the values of the next token will be 0 cause of the traingular form of the mask, this will tell Softmax that we don't want to take the any information of future tokens in order to be able to predict it using the past tokens only.\
So in the code we first create the mask using numpy and given the dimensions of our block size (8,8).\
Then using again numpy we can do this multiplication that I've talked about earlier to assing the values of wei where the ones are and the positions without 1 we fill it with negative infinite(in this case I'll use negative 1 billon just to make sure that I don't will fine any computer error down the road,which it could be the case if I leave it with negative infinite).


Section 5: Now that we are sure that the model will not cheat we can just do the Softmax.


Section 6: Here because the softmax already tell how many percentage of each head we want to take to have a good prediction we just do a matmul between how many of the values we want and the respective values of each head.


Section 7: Now that we got our values we gonna transpose and reshape them in order to have the same "form" of the tensor x that was our input to the function,to make things easier for future functions that it going to pass through.


Section 8: Finally and now that we got our results of the multihead atenttion we can "mix them".\
Every head you can see them as a proffesional in differents aspects of the tokens,one head can be an proffesional in semantic,other head and professional in sustantives,etc.\
So our output tensor will be just a concatenation of different proffesional opinions of what will be the next token.\
The heads are independent so the concatenation will sound like different peoples doing the same project and just combining they parts to the same project.This matmul will prevent that,it will "mix" the results (the opinion of each proffesional) to make the final output sound more coherent.


Section 9: While doing the backpropagation of the MHA I learned that I'll need the cache of the MHA ,so I put this section later in the creation of the model because without this information that I saved in the cache variable I can't do the backpropagation of the MHA.\
Is not the first time that I save a cache variable,for example in the CNN from scratch in the maxpool function you need to save the cache to know how was the size before to make the tensor of the gradient become a tensor with the correct size and dimensions.

In [1]:
def multi_head_atenttion(x, Wq, Wk, Wv, Wo, heads):

    # Section 0
    
    batch_size, block_size, total_embeddings = x.shape # The classic (B, T, C)

    head_size = total_embeddings // heads 

    
    # Section 1
    
    Q = x @ Wq

    K = x @ Wk

    V = x @ Wv

    
    # Section 2
    
    Q = Q.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    K = K.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    V = V.reshape(batch_size, block_size, heads, head_size).transpose(0, 2, 1, 3)

    
    # Section 3

    wei = Q @ K.transpose(0, 1, 3, 2)

    wei = wei * (head_size**-0.5) # More fancy than "(1 / np.sqrt(head_size))" which is also valid.


    
    # Section 4

    mask = np.tril(np.ones((block_size, block_size)))

    wei = np.where(mask == 0, -1e9, wei) #Underflow instead of -inf.


    
    # Section 5 

    probs = Softmax(wei)



    # Section 6

    out_heads = probs @ V


    
    # Section 7

    out_concat = out_heads.transpose(0, 2, 1, 3).reshape(batch_size, block_size, total_embeddings)


    
    # Section 8

    out = out_concat @ Wo



    # Section 9

    cache = (x, Wq, Wk, Wv, Wo, Q, K , V, wei, probs, out_concat, head_size)

    return out,cache

Initialization of the Parameters Gamma and Beta,which we will use in order to save and train them to make a more accurate model.\
The gamma parameter represents the Standard Deviation,which will be a tensor2D full of 1s (128,1),this can't be a tensor2D of zeros because if we multiply the information by 0 obviously the result will be 0 and we "kill the information".The first time it will pass as it is (because of the multiplication by 1)and then the parameter gamma will get actualize in order to pay more attention to the small values and have a more accurate standard deviation.\
The beta parameter represents the ideal mean that we want,it will start as a tensor2D full of 0s with the same size as the gamma tensor(128,1).

In [ ]:
gamma = np.ones((n_embd,))
beta = np.zeros((n_embd,))

Now that we have the "opinions" after we pass through the MHA we need to normalize them,because right now we can have for example: a value that is 50 and a value that is 0.01,here what is going to happen is that the higher value (50) will overlap the lower value (0.01).\
This is something that we don't want because both value are usefull,so in order to avoid this we will normalize them using a simple formula which is:\
input normalized is the substraction of the mean of the inputs to the inputs divided by the standard deviation of the inputs(the epsilon is as always just for the PC to take tiny numbers and don't convert them to 0 which is what the PC tends to do).

You can see x - x_mean like the process of making the numbers go lower, this a type of normalization like we do in the Softmax function but here instead of the Maximum number we use the Mean number.\
Then dividing with the standard deviation is the process to make them be "closer to each other",for example if they were scattered from 0 to 500 this will make them be scattered from 0 to 1.\
This is something that you will find in every layer norm function because is something standard in all of them,the things that changes in every model tho is the gamma and beta parameters because is something that is trained as every weight in the model,changing the specific deviation and mean in each model.(We need to save the cache for the manual backpropagation of the layer norm where we gonna give to gamma and beta their respective gradients)


PD: while I was learning about the history of this I come across something.\
The LN(Layer Norm)previously when the paper "Attention is all you need" came out,the LN was only used in the post-MHA,now in modern times the LN is used pre-MHA,models like GPT-2,GPT-3,Llama use the LN before the MHA,this is because doing it before the MHA makes the model easier to train(more stable).\
This doesn't change the explanation neither the formula but it changes what values the function takes and when you gonna use it in the training pipeline.

In [ ]:
def layer_norm(x,gamma,betta, eps=1e-5):

    mean = x.mean(axis = -1, keepdims=True)

    var = x.var(axis = -1, keepdims=True)

    std = np.sqrt(var + eps)

    
    
    x_hat = (x - mean) / std


    out = gamma * x_hat + beta


    cache = (x_hat,std,gamma)# All in the same variable,later we will unpack it.

    
    return out,gamma

This is just a "Residual Connection",our inputs (x) will travel across the entire transformer and after almost every function we gonna call this function that is just a addition of the previous inputs with the output of the function that have used those inputs.\
This will help with the gradient vanish cause by the multiplicatiion and all this function.\
As an anology you can imagine a signal(the inputs "x") that need to travel across 12 floors of a building(the transformer) at the time that the signal have reach the 12th floor is almost inexistent because in every floor it gets weaken,so to avoid this we enrich the signal in every floor to be able to reach the top of the building.



This function is so extra I know,but is for me to learn it and to have a more clean "memory map" of what is happening in the model.

In [ ]:
def residual_connection(x,functionout_x):
    return x + functionout_x

Here we initialize our parameters for the Feed_Foward Network(FFN):\
To create the weights we create a tensor2D of random values(W1) which will have the amount of rows of the number of embeddings of the tensor that comes from below (128 embds)and the amount of neurons that we want (512 neurons(128 * 4)).\
Why "*4"? Because that's the sweet spot that the creators found in the "Attention is all you need" paper to do this expansion and compression in the FFN.

For W2 we need to do the same but with the opossite values for the "columns & rows" of the tensor2D of the hidden layer of our FFN,comparison:\
W1 = (128,512)\
W2 = (512,128)\
Then we just use again another activation formula,and in this case I'll use He init for the hidden layer weigths because it is better for doing ReLU.\
And Xavier for the output layer weigths because after we do the residual connection using the output of the FFN function we going to do a Softmax and for Softmax is better(actually just standard idk if is better at this point) Xavier for it.

Then for the Biases as we know we need a Bias per Neuron so that what we do,we create a tensor1D of (1,512) for the B1.\
And a Tensor1D of (1,128) for the B2.

In [ ]:
W1 = np.random.randn(n_embd,n_embd * 4) * np.sqrt(2 / n_embd)
W2 = np.random.randn(n_embd * 4,n_embd) * np.sqrt(1 / (n_embd * 4))
B1 = np.zeros((1, n_embd * 4))
B2 = np.zeros((1, n_embd))

Here we have the Feed-Foward Network, in comparison we can see this part as the Neural Network of MNIST.\
What we have here is the Input that comes from below (Pos_encoding,MHA,LayerNorm,Residual_Connection) going to the hidden layer and "mixxing" with the W1 & B1,this creates Z1.\
Then Z1 does a ReLU and we get the activation of Z1 which is A1.\
Then A1 can do the same that previously which is "mixxing" but now with W2 & B2 resulting in Z2.\
Finally we return the values and if you notice it we don't do the A2 that we normally have to do, because before that we need to do the residual connection of this output of the function to the input of this function to just then get the A2, which will be Z2 enriched by the inputs and the result pass it through Softmax.

For a more In-Deep Explanation of what is happening in this I encourage you to see my Neural Network from Scratch repository where I explain this better.

In [ ]:
def FFN(x,W1,B1,W2,B2):

    Z1 = x @ W1 + B1
    
    A1 = Z1.ReLU()

    Z2 = A1 @ W2 + B2

    return Z1,A1,Z2

This is the Cross_Entropy_Loss function.\
It will receive the inputs which is the Tensor2D after the Softmax of the residual connection between the input and output of the FFN function, and the targets which is have the actual token that the model should have predict.

First we take the shape of our inputs and then collapse the batch_size and the block_size doing a reshape to it,ending up with a (32,65)tensor,which is 32 predictions of the 65 classes that the model could predict.

Then we need the one hot of the targets to be able to compare them with the inputs,so in order to do that we create a Tensor2D with the same size as the inputs but filled with zeros.\
Now using the fancy indexing technique we create a "ghost tensor" on top of our Tensor of zeros selecting the place where we have a number and once selected with the fancy indexing we replace that zero with a 1.0,creating a One_Hot with the places where was a number in the original targets now they have a 1.0.\
Finally we transform that numpy Tensor2D to a Tensor of the minitorch declaring that it doesn't need a gradient.

Now we can have the camparison between the inputs tensor and the targets_onehot tensor and store them in a variable.

Then we can do a Log() to that to increase the "punish" to the model on every error,but the log alone will make the numbers negative when we actually need them as positive numbers,the solution is just multiply them by -1.0 which will modify the symbol and conserving the number as it is.

Once we got the logs we just need to do the mean of that,my minitorch doesn't have the mean function right now so I'll use a trick with the Sum() that we did before which is divide the logs by the amount of predictions giving us the mean.

In [3]:
def cross_entropy_loss(inputs,targets):

    batch_size,block_size,classes = inputs.shape

    
    inputs_flat = inputs.reshape(batch_size * block_size,classes)

    
    oh_mask = np.zeros((batch_size * block_size,classes))

    oh_mask[np.arange(batch_size * block_size),targets.astype(int)] = 1.0 #Fancy indexing for the One Hot

    targets_oh = Tensor(oh_mask,requires_grad=False)


    correct_probs = (inputs_flat * targets_oh).Sum()

    
    logs = correct_probs.Log() * -1.0

    
    return logs * (1.0 / (batch_size * block_size)) 

Again create a function for this is actually not needed,but to learn how is the process that the model have to do I'll create this function and call it when needed.

Because the derivative of the addition of x + y is 1 the gradient just get distributed to both of them.\
Like in the foward pass the residual connection have the same functionality which is "amplify the signal" in order to avoid the gradient vanish,because without the residual connection amplifying the signal if the backprop of the FFN (for example) pass the gradient to the backprop of the layer norm if some of the weights/data are too low, the backprop of the layer norm will make it lower and so on; Pushing the gradient to become 0 and avoiding the correct backpropagation making us that we can't train the model because the gradient ended up as nothing mid backpropagation.

In [ ]:
def backprop_residual_connection(grad_input,grad_output):
    return grad_input + grad_output    

This is the backpropagation of the layer norm.\
We will take the gradient that comes from above and the cache that we have store earlier in the layer norm function of the foward pass.\
Next step is take the variables stored inside the cache variable and the shape of the gradient.

if you see the mathematical operation that we did in the foward pass to add the gamma and beta parameters in the layer normalization you will see that the operation is similar to a layer in a NN (X @ W + B) but in this case was (X * gamma + beta) so,in therms of beta is pretty simple,to get its derivative you just need to do the same thing that you do to get the derivative of a bias,which is the summatory of all the error.\
Then the derivative of gamma is conceptually similar,the difference is that in the NN you get a dot product and here you do a multiplication element by element,so the derivative will be the summatory of the comparison of the gradient with the x_hat.

Now dx is similar to get A1 in a NN the loss multiplied by the weights(in this case the parameter gamma).\
Next is "dx" which is the derivative to the input of the layer norm and honestly is just a mathematical formula that we traslate to code.If you want to know more about the formula you can check it out in this link: "https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html" (copy and paste it in the browser without the quotes if it doesn't work).\
I'll not explain the formula because is pure mathematical logic,but to understand what is happening without the need of the link you can imagine that because is the layer_norm a process in the middle of the transformer it will have a previous function and in order to continue the backpropagation we need to keep passing the gradient and thanks to that formula the gradient can reach the input of the function and continue to the next backprop_function.

In [ ]:
def backprop_LayerNorm(grad_output,cache):

    x_hat, std, gamma = cache

    batch_size, block_size, n_embd = grad_output.shape


    dbeta = np.sum(grad_output, axis=(0,1))

    dgamma = np.sum(grad_output * x_hat, axis=(0,1))



    dx_hat = grad_output * gamma

    dx = (1.0 / n_embd) * (1.0 / std) * (
        n_embd * dx_hat -
        np.sum(dx_hat, axis = -1, keepdims = True) -
        x_hat * np.sum(dx_hat * x_hat, axis = -1, keepdims = True)
    )


    return dx, dbeta, dgamma

This is the "Final Boss of the Backpropagation"(honestly I felt more difficult the backprop of the layernorm because of its mathematics in dOut,this is practically just a "dance" of dimensions).\
Because of my previous creation of the Autograd engine, I learned a lot of the back propagation of the different formulas,here we use  in a huge part the bakpropagation of the matmul which is:\
if Z = X @ Y \
The derivative of X is = grad_output @ Y.T \
And the derivative of Y is = X.T @ grad_output\
If you want to memorize this is pretty simple, just put your tensor of gradients in the same place of the foward of the variable that you want the derivative and multiply it by the other variable transpose.

In [ ]:
def backprop_MHA(grad_output, cache):

    # Section 1
    
    x, Wq, Wk, Wv, Wo, Q, K, V, wei, probs, out_concat, head_size = cache

    batch_size, block_size, n_embd = x.shape

    heads_amount = Q.shape[1]


    # Section 2

    dWo
    

    
    return